In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
   for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/processing-tfl-cycle-hire-data/bike_df.csv
/kaggle/input/processing-tfl-cycle-hire-data/__results__.html
/kaggle/input/processing-tfl-cycle-hire-data/__notebook__.ipynb
/kaggle/input/processing-tfl-cycle-hire-data/__output__.json
/kaggle/input/processing-tfl-cycle-hire-data/custom.css


# Load

In [4]:
csv_path = "../input/processing-tfl-cycle-hire-data/bike_df.csv"
bike_df = pd.read_csv(csv_path)
bike_df.head()


bike_df.set_index('RentalID', inplace=True)
#bike_df.drop('Unnamed: 0', axis=1, inplace=True)
bike_df.sort_index(inplace=True)

bike_df.head()

#Format date
bike_df[['StartDateTime','EndDateTime']] = bike_df[['StartDateTime','EndDateTime']].apply(pd.to_datetime, format='%Y-%m-%d %H:%M:%S')
bike_df.head()

bike_df.dtypes



,RentalID,BikeID,StartDateTime,StartStationID,StartStation,EndDateTime,EndStationID,EndStation,Duration_Seconds
0,57571323,1004,2016-08-24 00:00:00,210,"Hinde Street, Marylebone",2016-08-24 00:19:00,261,"Princes Square, Bayswater",1140
1,57571324,8925,2016-08-24 00:00:00,477,"Spindrift Avenue, Millwall",2016-08-24 00:21:00,554,"Aberfeldy Street, Poplar",1260
2,57571325,3100,2016-08-24 00:00:00,188,"Nutford Place, Marylebone",2016-08-24 00:18:00,265,"Southwick Street, Paddington",1080
3,57571326,11247,2016-08-24 00:00:00,273,"Belvedere Road, South Bank",2016-08-24 00:18:00,246,"Berry Street, Clerkenwell",1080
4,57571327,256,2016-08-24 00:00:00,578,"Hollybush Gardens, Bethnal Green",2016-08-24 00:42:00,146,"Vauxhall Bridge , Pimlico",2520


BikeID                       int64
StartDateTime       datetime64[ns]
StartStationID               int64
StartStation                object
EndDateTime         datetime64[ns]
EndStationID                 int64
EndStation                  object
Duration_Seconds             int64
dtype: object

# Filtering data
I only want three stations nearest the site, all in Vauxhall.

In [8]:
startview_df = bike_df[bike_df['StartStation'].str.contains('Vauxhall')]
startview_df.sort_values(by=['StartStation'])

endview_df = bike_df[bike_df['EndStation'].str.contains('Vauxhall')]
endview_df.sort_values(by=['EndStation'])

start_stations = pd.DataFrame({'StartStation': startview_df.StartStation.unique()})
#And then just get the list of all unique subreddits they are active in, assigning it to a new column:

end_stations = pd.DataFrame({'EndStation': endview_df.EndStation.unique()})

start_stations['StartStationIDs'] = [list(set(startview_df['StartStationID'].loc[startview_df['StartStation'] == x['StartStation']])) 
    for _, x in start_stations.iterrows()]

end_stations['EndStationIDs'] = [list(set(endview_df['EndStationID'].loc[endview_df['EndStation'] == x['EndStation']])) 
    for _, x in end_stations.iterrows()]

print(start_stations)

print(start_stations,'\n\n',end_stations)

,BikeID,StartDateTime,StartStationID,StartStation,EndDateTime,EndStationID,EndStation,Duration_Seconds
RentalID,,,,,,,,
100037818,17800,2020-07-29 18:59:00,818,"Westminster Pier, Westminster",2020-07-29 19:07:00,100,"Albert Embankment, Vauxhall",480
80217341,5397,2018-09-11 17:54:00,601,"BBC White City, White City",2018-09-11 18:28:00,100,"Albert Embankment, Vauxhall",2040
93849830,8992,2019-12-22 10:57:00,100,"Albert Embankment, Vauxhall",2019-12-22 11:41:00,100,"Albert Embankment, Vauxhall",2640
91866769,13582,2019-10-04 15:02:00,183,"Riverlight North, Nine Elms",2019-10-04 16:06:00,100,"Albert Embankment, Vauxhall",3840
99578687,583,2020-07-18 01:02:00,87,"Devonshire Square, Liverpool Street",2020-07-18 01:24:00,100,"Albert Embankment, Vauxhall",1320
...,...,...,...,...,...,...,...,...
85962139,10207,2019-04-17 19:34:00,440,"Kennington Oval, Oval",2019-04-17 19:40:00,91,"Walnut Tree Walk, Vauxhall",360
85962683,2616,2019-04-17 19:49:00,386,"Moor Street, Soho",2019-04-17 20:05:00,91,"Walnut Tree Walk, Vauxhall",960
85962687,16497,2019-04-17 19:49:00,55,"Finsbury Circus, Liverpool Street",2019-04-17 20:08:00,91,"Walnut Tree Walk, Vauxhall",1140


                             StartStation StartStationIDs
0   Kennington Lane Rail Bridge, Vauxhall           [270]
1       New Spring Gardens Walk, Vauxhall           [813]
2                  Lambeth Road, Vauxhall           [139]
3             Fire Brigade Pier, Vauxhall           [387]
4               Vauxhall Bridge , Pimlico           [146]
5             Black Prince Road, Vauxhall           [310]
6            South Lambeth Road, Vauxhall           [600]
7                Lollard Street, Vauxhall           [117]
8              Walnut Tree Walk, Vauxhall            [91]
9                Vauxhall Cross, Vauxhall            [74]
10        Kennington Lane Tesco, Vauxhall           [305]
11              Sancroft Street, Vauxhall            [86]
12            Kennington Road  , Vauxhall           [235]
13                     Oval Way, Vauxhall           [355]
14                Vauxhall Walk, Vauxhall           [437]
15            Albert Embankment, Vauxhall           [100]
16            

The stations I have want to reference are: New Spring Gardens Walk, Vauxhall (813), Vauxhall Cross, Vauxhall (074) and Kennington Lane Rail Bridge, Vauxhall (270) as these are the three closest to the monitoring station.

In [10]:
station_list=[813,74,270]

#Filter bike database by the start station if it matches the list
start_mask =bike_df['StartStationID'].isin(station_list) & ~bike_df['EndStationID'].isin(station_list)
end_mask =bike_df['EndStationID'].isin(station_list) & ~bike_df['StartStationID'].isin(station_list)
within_mask =bike_df['StartStationID'].isin(station_list) & bike_df['EndStationID'].isin(station_list)

start_journeys = bike_df[start_mask]
end_journeys = bike_df[end_mask]
within_journeys = bike_df[within_mask]

#Then combine the two
frames = [start_journeys, end_journeys, within_journeys]

filtered_bike = pd.concat(frames)
filtered_bike.head()
#print('Number of journeys starting at Vauxhall: ',len(start_journeys),'\n Number of journeys ending in Vauxhall: ',len(end_journeys),'\n Total number of journeys: ',len(filtered_bike))

filtered_bike =filtered_bike.loc[(filtered_bike['StartDateTime'] >= '2018-01-01 00:00:00') & (filtered_bike['EndDateTime'] <= '2020-12-31 00:00:00')]
filtered_bike

num_duplicates = len(filtered_bike)-len(filtered_bike.drop_duplicates())
num_duplicates

#filtered_bike = filtered_bike.drop_duplicates()


,BikeID,StartDateTime,StartStationID,StartStation,EndDateTime,EndStationID,EndStation,Duration_Seconds
RentalID,,,,,,,,
57571411,3553,2016-08-24 00:20:00,270,"Kennington Lane Rail Bridge, Vauxhall",2016-08-24 00:23:00,440,"Kennington Oval, Oval",180
57571469,1586,2016-08-24 00:59:00,813,"New Spring Gardens Walk, Vauxhall",2016-08-24 01:27:00,305,"Kennington Lane Tesco, Vauxhall",1680
57571804,9241,2016-08-24 03:08:00,270,"Kennington Lane Rail Bridge, Vauxhall",2016-08-24 03:16:00,602,"Union Grove, Wandsworth Road",480
57571984,2750,2016-08-24 05:29:00,813,"New Spring Gardens Walk, Vauxhall",2016-08-24 05:39:00,139,"Lambeth Road, Vauxhall",600
57571987,4546,2016-08-24 05:30:00,813,"New Spring Gardens Walk, Vauxhall",2016-08-24 05:39:00,139,"Lambeth Road, Vauxhall",540


,BikeID,StartDateTime,StartStationID,StartStation,EndDateTime,EndStationID,EndStation,Duration_Seconds
RentalID,,,,,,,,
72337793,5804,2018-01-01 00:19:00,74,"Vauxhall Cross, Vauxhall",2018-01-01 00:33:00,664,"Austin Road, Battersea Park",840
72337795,5277,2018-01-01 00:20:00,74,"Vauxhall Cross, Vauxhall",2018-01-01 00:33:00,664,"Austin Road, Battersea Park",780
72337819,9558,2018-01-01 00:22:00,813,"New Spring Gardens Walk, Vauxhall",2018-01-01 00:54:00,714,"Stewart's Road, Wandsworth Road",1920
72337823,11769,2018-01-01 00:22:00,813,"New Spring Gardens Walk, Vauxhall",2018-01-01 00:53:00,714,"Stewart's Road, Wandsworth Road",1860
72337867,6975,2018-01-01 00:24:00,74,"Vauxhall Cross, Vauxhall",2018-01-01 00:44:00,826,"Allington street, Westminster",1200
...,...,...,...,...,...,...,...,...
104714876,16586,2020-12-27 16:22:00,74,"Vauxhall Cross, Vauxhall",2020-12-27 19:10:00,270,"Kennington Lane Rail Bridge, Vauxhall",10080
104720294,6723,2020-12-28 07:38:00,270,"Kennington Lane Rail Bridge, Vauxhall",2020-12-28 07:39:00,270,"Kennington Lane Rail Bridge, Vauxhall",60
104737033,19570,2020-12-29 12:35:00,74,"Vauxhall Cross, Vauxhall",2020-12-29 12:36:00,74,"Vauxhall Cross, Vauxhall",60


# Pivoting
hourly,avg length of rides

# Export